In [1]:
from sklearn.decomposition import PCA
from PIL import Image

#读取视频文件
def loadmkv(file,img_num=10):
    import cv2
    #标注处理
    labels_list = []
    i = 1
    with open(file + '_steering.csv', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            labels_list.append(row['wheel'])
            if (i>=img_num):
                break
            i = i + 1
    #图像处理        
    cap = cv2.VideoCapture(file +'_front.mkv')
    ret = True
    frame_list = []
    i = 1 
    while(ret == True):
        ret, frame = cap.read()
        if (ret == True):
            
            frame_list.append(frame)
            del frame
            if (i>=img_num):
                break
            i = i + 1
            #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #cv2.imshow('frame',gray)
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    
    return ((len(frame_list),frame_list),(len(labels_list),labels_list))

In [6]:
import pickle
from os.path import isfile, isdir
import csv

epochs_path = '../epochs/'
outputf_path = '../output/'
for i in range(1,2):
    filename = epochs_path + 'epoch0' + str(i)
    outputfilename = outputf_path + 'epoch0' + str(i)
    if not isfile(outputfilename + '.p'):
        features,labels = loadmkv(filename,3)
        with open(outputfilename + '.p', 'wb') as outputfile:
            pickle.dump((features,labels), outputfile)
        del features,labels
        
print('load file completion')

load file completion


In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        x,y = pickle.load(fo, encoding='bytes')
    return x,y

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing import image
import numpy as np
from resnet50 import ResNet50

x,y = unpickle('../output/epoch01.p')

model = ResNet50(include_top=True, weights=None,input_shape=(720,1280,3))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
x_train = x
y_train = y

model.fit(x_train, y_train,
          epochs=10,
          batch_size=10)
#score = model.evaluate(x_test, y_test, batch_size=10)
print('Score:', score)

TypeError: Error when checking model input: data should be a Numpy array, or list/dict of Numpy arrays. Found: (3, [array([[[156, 132, 136],
        [157, 133, 137],
        [157, 133, 137],
        ..., 
        [249, 225, 247],
        [249, 225, 247],
        [249, 225, 247]],

       [[156, 132, 136],
    ...